In [1]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import yfinance as yf
from tqdm import tqdm

In [2]:
df = pd.read_csv('.\s&p500\S&P 500 Historical Components & Changes(04-08-2024).csv')

In [3]:
df.head()

,date,tickers
0,1996-01-02,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
1,1996-01-03,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
2,1996-01-04,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
3,1996-01-10,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
4,1996-01-11,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."


2000/01/01~2022/12/31까지 존재한 모든 주식을 열로 가지는 새로운 dataframe 생성

In [4]:
# Convert the 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Filter the data for the specified date range
filtered_data = df[(df['date'] >= '2006-01-01') & (df['date'] <= '2022-12-31')]

# Extract unique tickers over the filtered date range
unique_tickers = set()
for tickers in filtered_data['tickers']:
    unique_tickers.update(tickers.split(','))

# Create a new dataframe with the date range from 2006-01-01 to 2022-12-31
date_range = pd.date_range(start='2006-01-01', end='2022-12-31', freq='D')
new_df = pd.DataFrame(date_range, columns=['date'])
# Add a new column S&P500, SPX .INX, ^GSPC' after the 'date' column
new_df['^GSPC'] = np.nan
# Add '^GSPC' to the set of unique tickers
unique_tickers.add('^GSPC')
# Merge the new dataframe with the existing dataframe of unique tickers
new_df = pd.concat([new_df, pd.DataFrame(columns=sorted(unique_tickers))], axis=1)

In [5]:
new_df.head()

,date,^GSPC,A,AABA,AAL,AAP,AAPL,ABBV,ABC,ABI,...,XRAY,XRX,XTO,XYL,YUM,ZBH,ZBRA,ZION,ZTS,^GSPC
0,2006-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2006-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2006-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2006-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2006-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
unique_tickers

{'A',
 'AABA',
 'AAL',
 'AAP',
 'AAPL',
 'ABBV',
 'ABC',
 'ABI',
 'ABKFQ',
 'ABMD',
 'ABS',
 'ABT',
 'ACAS',
 'ACGL',
 'ACN',
 'ACS',
 'ACV',
 'ADBE',
 'ADCT',
 'ADI',
 'ADM',
 'ADP',
 'ADS',
 'ADSK',
 'ADT',
 'AEE',
 'AEP',
 'AES',
 'AET',
 'AFL',
 'AGN',
 'AIG',
 'AIV',
 'AIZ',
 'AJG',
 'AKAM',
 'AKS',
 'ALB',
 'ALGN',
 'ALK',
 'ALL',
 'ALLE',
 'ALTR',
 'ALXN',
 'AMAT',
 'AMCC',
 'AMCR',
 'AMD',
 'AME',
 'AMG',
 'AMGN',
 'AMP',
 'AMT',
 'AMZN',
 'AN',
 'ANDV',
 'ANDW',
 'ANET',
 'ANF',
 'ANRZQ',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'APC',
 'APCC',
 'APD',
 'APH',
 'APOL',
 'APTV',
 'ARE',
 'ARG',
 'ARNC',
 'ASH',
 'ASN',
 'ASO',
 'AT',
 'ATGE',
 'ATI',
 'ATO',
 'ATVI',
 'AV',
 'AVB',
 'AVGO',
 'AVP',
 'AVY',
 'AW',
 'AWK',
 'AXP',
 'AYE',
 'AYI',
 'AZO',
 'BA',
 'BAC',
 'BALL',
 'BAX',
 'BBBY',
 'BBT',
 'BBWI',
 'BBY',
 'BC',
 'BCR',
 'BDK',
 'BDX',
 'BEAM',
 'BEN',
 'BF.B',
 'BHF',
 'BHGE',
 'BIG',
 'BIIB',
 'BIO',
 'BJS',
 'BK',
 'BKNG',
 'BKR',
 'BLK',
 'BLL',
 'BLS',
 'BMC'

In [7]:
# Function to fetch and update the closing prices for a given ticker
def update_ticker_data(ticker, df):
    try:
        # Fetch historical market data
        ticker_data = yf.download(ticker, start='2006-01-01', end='2022-12-31')
        ticker_data = ticker_data['Close']
        
        # Update the dataframe with the closing prices
        for date in ticker_data.index:
            if date in df['date'].values:
                df.loc[df['date'] == date, ticker] = ticker_data[date]
    except Exception as e:
        print(f"Failed to fetch data for {ticker}: {e}")

# Update the dataframe with the closing prices for each unique ticker
for ticker in sorted(unique_tickers):
    update_ticker_data(ticker, new_df)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AABA']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ABC']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ABI']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2006-01-01 -> 2022-12-31)')


$ABI: possibly delisted; No price data found  (1d 2006-01-01 -> 2022-12-31)


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ABKFQ']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ACAS']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2006-01-01 -> 2022-12-31)')


$ACAS: possibly delisted; No price data found  (1d 2006-01-01 -> 2022-12-31)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ADS']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*****

...

[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['WFT']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['WIN']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['WLTW']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['WPX']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 

$WYE: possibly delisted; No price data found  (1d 2006-01-01 -> 2022-12-31)


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['WYND']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['XEC']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['XL']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['XLNX']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 

$XTO: possibly delisted; No price data found  (1d 2006-01-01 -> 2022-12-31)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [8]:
new_df.head()

,date,^GSPC,A,AABA,AAL,AAP,AAPL,ABBV,ABC,ABI,...,XRAY,XRX,XTO,XYL,YUM,ZBH,ZBRA,ZION,ZTS,^GSPC
0,2006-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2006-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2006-01-03,1268.800049,23.962805,NaN,37.450001,43.52,2.669643,NaN,NaN,NaN,...,27.014999,39.28854,NaN,NaN,16.804457,66.485435,42.830002,76.480003,NaN,1268.800049
3,2006-01-04,1273.459961,24.027182,NaN,39.200001,43.830002,2.6775,NaN,NaN,NaN,...,27.264999,39.235836,NaN,NaN,16.948238,67.116508,42.41,77.019997,NaN,1273.459961
4,2006-01-05,1273.479980,24.656652,NaN,39.709999,44.040001,2.656429,NaN,NaN,NaN,...,26.955,38.814228,NaN,NaN,17.67074,66.407768,42.529999,77.720001,NaN,1273.47998


In [11]:
# Save the updated dataframe to a CSV file
output_file_path = '.\s&p500\s&p500_stock_data.csv'
new_df.to_csv(output_file_path, index=False)